In [ ]:
import pandas as pd
import unicodedata
import json
import json
from datetime import date

# Función para normalizar nombres de canciones (quita espacios extra y pasa a minúsculas)
def normalize(song):
    song = song.strip().lower()
    song = ''.join(c for c in unicodedata.normalize('NFKD', song) if not unicodedata.combining(c) or c == 'ñ')
    return song

# Cargar los archivos JSON
with open(r'C:\Users\Yo\Desktop\setlist\SetlistFM-Setlist-Evolution\album_data\Los Piojos.json', 'r', encoding='utf-8') as f:
    albums_data = json.load(f)


with open('Setlists_Piojos.json', 'r', encoding='utf-8') as f:
    setlists_data = json.load(f)

# Crear diccionarios para mapear cada canción a su álbum y a su color
song_to_album = {}
song_to_color = {}
for album in albums_data.get("albums", []):
    album_title = album.get("title", "Ineditos")
    color = album.get("color", "gray")
    for track in album.get("tracks", []):
        norm_track = normalize(track)
        # Si la canción aparece en varios álbumes, se asigna el primero encontrado
        if norm_track not in song_to_album:
            song_to_album[norm_track] = album_title
            song_to_color[norm_track] = color

# Crear un diccionario que mapea cada álbum a su color en formato HEX
album_to_color = {}
for album in albums_data.get("albums", []):
    album_to_color[album.get("title", "Ineditos")] = album.get("color", "#39df00")

df_popularidad = pd.read_excel(r'C:\Users\Yo\Desktop\setlist\SetlistFM-Setlist-Evolution\canciones_popularidad.xlsx')


# Ordenar los setlists por fecha (usando year, month, day)
setlists_data.sort(key=lambda s: date(s["year"], s["month"], s["day"]))

# Procesar cada setlist y contar acumulativamente
records = []
cumulative_counts = {}

for setlist in setlists_data:
    # Crear fecha en formato ISO
    current_date = date(setlist["year"], setlist["month"], setlist["day"]).isoformat()
    
    # Procesar la lista de canciones, separando por "/" si es necesario
    canciones_raw = setlist.get("songs", [])
    canciones = []
    for item in canciones_raw:
        for part in item.split('/'):
            if part.strip():
                canciones.append(part.strip())
    
    # Actualizar los conteos acumulados
    for song in canciones:
        norm_song = normalize(song)
        cumulative_counts[norm_song] = cumulative_counts.get(norm_song, 0) + 1
    # Registrar el estado acumulado hasta la fecha actual
    for song, count in cumulative_counts.items():
        album = song_to_album.get(song, "Ineditos")
        color = song_to_color.get(song, "#000000")
        records.append({
            "fecha": current_date,
            "canción": song.title(),  # Capitalizamos para visualización
            "álbum": album,
            "color": color,
            "veces_tocada": count,

        })

# Crear DataFrame y agrupar para quedarnos con el último conteo de cada fecha
df_vuelta_total = pd.DataFrame(records)


df_vuelta_total[df_vuelta_total['fecha'] == '2025-05-25'].head()

# Asume que song_to_album y song_to_color ya están definidos


all_records = []

for setlist in setlists_data:
    current_date = date(setlist["year"], setlist["month"], setlist["day"]).isoformat()
    canciones_raw = setlist.get("songs", [])
    canciones = []
    for item in canciones_raw:
        for part in item.split('/'):
            if part.strip():
                canciones.append(part.strip())
    for song in canciones:
        norm_song = normalize(song)
        album = song_to_album.get(norm_song, "Ineditos")
        color = song_to_color.get(norm_song, "#000000")
        all_records.append({
            "fecha": current_date,
            "canción": song.title(),
            "álbum": album,
            "color": color  # Solo cuenta 1 para este concierto
        })

# Super DataFrame con todas las fechas y canciones (sin acumulativo)
df_super = pd.DataFrame(all_records)



In [ ]:
import json
from datetime import date

# Asume que song_to_album y song_to_color ya están definidos

with open('setlists_vuelta.json', 'r', encoding='utf-8') as f:
    setlists_data = json.load(f)

all_records = []

for setlist in setlists_data:
    current_date = date(setlist["year"], setlist["month"], setlist["day"]).isoformat()
    canciones_raw = setlist.get("songs", [])
    canciones = []
    for item in canciones_raw:
        for part in item.split('/'):
            if part.strip():
                canciones.append(part.strip())
    for song in canciones:
        norm_song = normalize(song)
        album = song_to_album.get(norm_song, "Ineditos")
        color = song_to_color.get(norm_song, "#000000")
        all_records.append({
            "fecha": current_date,
            "canción": song.title(),
            "álbum": album,
            "color": color  # Solo cuenta 1 para este concierto
        })

# Super DataFrame con todas las fechas y canciones (sin acumulativo)
df_super = pd.DataFrame(all_records)
df_super



# 1. Normaliza los nombres de las canciones en ambos DataFrames para hacer el merge
df_super['cancion_norm'] = df_super['canción'].apply(normalize)
df_popularidad['cancion_norm'] = df_popularidad['canción'].apply(normalize)

# 2. Une la popularidad a cada canción tocada en cada concierto
df_super = df_super.merge(
    df_popularidad[['cancion_norm', 'indice_popularidad']],
    on='cancion_norm',
    how='left'
)

# 3. Calcula el promedio de popularidad por concierto (por fecha)
df_conciertos_pop = df_super.groupby('fecha')['indice_popularidad'].mean().reset_index()
df_conciertos_pop = df_conciertos_pop.sort_values('indice_popularidad', ascending=False)


# 4. Ahora puedes ver los conciertos más "mainstream" y más "alternativos"
print("Recitales ordenados de mayor a menor popularidad en promedio:")
print(df_conciertos_pop.sort_values('indice_popularidad', ascending=False))
df_conciertos_pop.to_excel('listas_vuelta_ordenadas_popularidad.xlsx')